### Betfair Historical Data large dataset creation
1. Following `2_betfair_advanced_data_test.ipynb` steps to download historical data
2. Iterating the stream through a list of files to create a dataset containing multiple events and race days


To connect to the Betfair API through betfairlightweight, you must have first set up your API certificates and saved the login details in dictionary format in a file called `api_logins.json` in the project home directory (`/betfair_project`). The following connects and should return `<LoginResource>` if successful.

In [1]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from pathlib import Path, PurePath #To define open and save locations that are cross-compatible between Windows/Linux
from bz2 import BZ2File #To unzip the Betfair data from its downloaded format
import matplotlib.pyplot as plt

project_dir = Path.cwd().parents[1]
logins_dir = project_dir / 'api_logins.json'

with open(logins_dir) as f:
    login_dict =  json.load(f)
    
trading = betfairlightweight.APIClient(username=login_dict['my_username'],
                                       password=login_dict['my_password'],
                                       app_key=login_dict['my_app_key'],
                                       certs=login_dict['certs_path'])

trading.login()

<LoginResource>

To perform the following, you must have purchased data from the [Betfair Historical data service](https://historicdata.betfair.com/#/home). Advanced data for all sports has been offered for free for Jan - May 2020. To understand more about what different packages of data include, look in the data dictionaries folder in this project. The following lists the data that you have purchased on your Betfair account. Since data is purchased by month, that is how it is represented here:

In [2]:
my_data = trading.historic.get_my_data()
for i in my_data:
    print(i)

{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-01-01T00:00:00', 'purchaseItemId': 42364}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-02-01T00:00:00', 'purchaseItemId': 42364}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-03-01T00:00:00', 'purchaseItemId': 42364}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-04-01T00:00:00', 'purchaseItemId': 41549}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-05-01T00:00:00', 'purchaseItemId': 41549}
{'sport': 'Horse Racing', 'plan': 'Basic Plan', 'forDate': '2016-01-01T00:00:00', 'purchaseItemId': 25202}
{'sport': 'Horse Racing', 'plan': 'Basic Plan', 'forDate': '2017-06-01T00:00:00', 'purchaseItemId': 770}
{'sport': 'Horse Racing', 'plan': 'Basic Plan', 'forDate': '2018-01-01T00:00:00', 'purchaseItemId': 24527}
{'sport': 'Horse Racing', 'plan': 'Basic Plan', 'forDate': '2018-02-01T00:00:00', 'purchaseItemId': 24527}
{'sport': 'Horse Racing'

The following selects just the data for from 01/01/20 to 01/03/20 and returns a dictionary of the contents of the data which we go on to use to download specific country/race/market types, and tells us the size of the entire selection:

In [3]:
collection_options = trading.historic.get_collection_options(
    "Horse Racing", "Advanced Plan", 1, 1, 2020, 1, 3, 2020
)

print(collection_options)

basket_size = trading.historic.get_data_size(
    "Horse Racing", "Advanced Plan", 1, 1, 2020, 1, 3, 2020
)
print(basket_size)

{'marketTypesCollection': [{'name': '', 'count': 1997}, {'name': 'ANTEPOST_WIN', 'count': 22}, {'name': 'DAILY_WIN_DIST', 'count': 1}, {'name': 'EACH_WAY', 'count': 1788}, {'name': 'FORECAST', 'count': 438}, {'name': 'MATCH_BET', 'count': 1692}, {'name': 'OTHER_PLACE', 'count': 4746}, {'name': 'PLACE', 'count': 9161}, {'name': 'RACE_WIN_DIST', 'count': 175}, {'name': 'REV_FORECAST', 'count': 784}, {'name': 'WIN', 'count': 11923}, {'name': 'WITHOUT_FAV', 'count': 306}], 'countriesCollection': [{'name': 'AE', 'count': 511}, {'name': 'AU', 'count': 11874}, {'name': 'ES', 'count': 6}, {'name': 'FR', 'count': 549}, {'name': 'GB', 'count': 10230}, {'name': 'HK', 'count': 1}, {'name': 'IE', 'count': 2147}, {'name': 'NZ', 'count': 2016}, {'name': 'SA', 'count': 44}, {'name': 'SG', 'count': 314}, {'name': 'US', 'count': 4037}, {'name': 'ZA', 'count': 1304}], 'fileTypeCollection': [{'name': 'E', 'count': 1997}, {'name': 'M', 'count': 31036}]}
{'totalSizeMB': 5731, 'fileCount': 33033}


To dig into particular dates, countries, race types and market types we use `.get_file_list()`, specifying our filtering parameters. For example, you can see above that within the sub-dictionary for `'marketTypesCollection'` we have `'WIN'`, among others. The following retrieves a list of the event files that match these parameters. We select only data for 01/02/20, for the `'WIN'` market only and in `'GB'`.

In [4]:
file_list = trading.historic.get_file_list(
    "Horse Racing",
    "Advanced Plan",
    from_day=1,
    from_month=2,
    from_year=2020,
    to_day=3,
    to_month=2,
    to_year=2020,
    market_types_collection=["WIN"],
    countries_collection=["GB"],
    file_type_collection=["M"]\
    ,
)
print(file_list)

TypeError: get_file_list() missing 6 required positional arguments: 'from_day', 'from_month', 'from_year', 'to_day', 'to_month', and 'to_year'

To downoad a file, `download_file()` is used, where we specify the download location (on Betfair) in `file_path` and the directory to which it is saved in `store_directory`. The filename is kept as is from Betfair. We both download the file and use the command to assign the file's location to the variable `download` so we can use it in subsequent operations. `file_path` below is a list input over which we iterate `.download_file()`.

In [5]:
data_dir = project_dir / 'data' / 'raw' / 'api'

available_files = file_list

print(available_files)

downloaded_files = [] #list of directories of each download

for file in available_files:
    download = trading.historic.download_file(file_path = file, store_directory = data_dir)
    print(download)
    downloaded_files.append(download)

['/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130661.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096552.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136214.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133543.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130666.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096559.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136219.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130671.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133548.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096566.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136224.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133553.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130676.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168130245.bz2', '/xds

/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168203080.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168208888.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168203085.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168208893.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168203090.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168208898.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168203095.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168208903.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168203100.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168264297.bz2
/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168264302.bz2

The downloaded file is .json formatted compressed in a bzip2 (.bz2) folder. The following extracts the file where it is downloaded. It is given no file extension, however it remains in .txt format and is readable in any text reader.

In [6]:
extracted_files = []

for file in downloaded_files:
    zipfile = BZ2File(file) # open the file
    data = zipfile.read() # get the decompressed data
    newfilepath = file.split('.bz2')[0] # removing the extension and saving without a filetype
    open(newfilepath, 'wb').write(data) # write an uncompressed file
    extracted_files.append(newfilepath)
    zipfile.close()
    
print(extracted_files)

['/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168130661', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168096552', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168136214', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168133543', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168130666', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168096559', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168136219', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168130671', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168133548', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168096566', '/Users/tombardrick/Documents/projects/betfair/betfair_project/data/raw/api/1.168136224', '/Users/t

## Using stream listener to read extracted data

We now have a readable raw text file in Betfair's dictionary format. To interpret the data with python, betfairlightweight includes a stream listener which will interpret the input of historical data in the same way that it does for live data. The below is edited from [here](https://github.com/liampauling/betfair/blob/104ff4cb8734038cb9351e74d16dc7bd018111bc/examples/examplestreaminghistorical.py). 

This finds the chosen variables in the data file and outputs them in a .csv format in output.txt. Note: output.txt is ignored by git and is not uploaded into the repository.

In [7]:
from betfairlightweight import StreamListener
from betfairlightweight.streaming.stream import MarketStream

class HistoricalStream(MarketStream):
    # create custom listener and stream

    def __init__(self, listener):
        super(HistoricalStream, self).__init__(listener)

    def on_process(self, market_books):       
        
        with open("output.txt", "a") as output:            
            for market_book in market_books:
                for runner in market_book.runners:
                    output.write(
                        "%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\n"
                        % (
                            market_book.publish_time, #datetime.datetime
                            market_book.market_id, #float
                            market_book.status, #unicode
                            market_book.inplay, #bool
                            market_book.market_definition.name, # str
                            runner.selection_id, #int
                            runner.last_price_traded or "", #float
                            runner.total_matched or "", #float
                            runner.sp.actual_sp or "", #float
                            runner.adjustment_factor or "", #float
                            runner.status,
                            market_book.total_matched or "", #float
                        )
                    )


class HistoricalListener(StreamListener):
    def _add_stream(self, unique_id, stream_type):
        if stream_type == "marketSubscription":
            return HistoricalStream(self)
        
listener = HistoricalListener(max_latency=None)

Iterating the Historical Listener through the list of `extracted_files`. For the full day of racing 01/02/20, `output.txt` is 493mb.

In [8]:
with open("output.txt", "w") as output:
    output.write("Time,MarketId,Status,Inplay,MarketName,SelectionId,LastPriceTraded,TotalMatched,Sp,AdjFactor,RunnerStatus,MktTotalMatched\n")

for file in extracted_files:
    stream = trading.streaming.create_historical_stream(
        directory=file,
        listener=listener,
    )
    stream.start()

Delete all downloaded files:

In [9]:
for file in downloaded_files:
    file = Path(file)
    file.unlink()
    
for file in extracted_files:
    file = Path(file)
    file.unlink()

We can now read output.txt with pandas.

In [10]:
df = pd.read_csv('output.txt')
df.head()

,Time,MarketId,Status,Inplay,MarketName,SelectionId,LastPriceTraded,TotalMatched,Sp,AdjFactor,RunnerStatus,MktTotalMatched
0,2020-01-31 11:41:45.697000,1.168131,OPEN,False,7f Hcap,25545675,NaN,NaN,NaN,5.95,ACTIVE,NaN
1,2020-01-31 11:41:45.697000,1.168131,OPEN,False,7f Hcap,25238892,NaN,NaN,NaN,30.53,ACTIVE,NaN
2,2020-01-31 11:41:45.697000,1.168131,OPEN,False,7f Hcap,24707172,NaN,NaN,NaN,7.01,ACTIVE,NaN
3,2020-01-31 11:41:45.697000,1.168131,OPEN,False,7f Hcap,24420205,NaN,NaN,NaN,8.45,ACTIVE,NaN
4,2020-01-31 11:41:45.697000,1.168131,OPEN,False,7f Hcap,26314798,NaN,NaN,NaN,33.12,ACTIVE,NaN


### Creating variables for better analysis

To understand the evolution of matches at each price, we can create `TradeSize` by looking at the change in `TotalMatched` grouped by `LastPriceTraded`.

In [11]:
df['TradeSize'] = df.groupby(['MarketId','SelectionId', 'LastPriceTraded'])['TotalMatched'].diff()

In order to make race timeseries data comparable between events, we need an indicator of what stage the market is in. For example, betting behaviour is very different when the market is first created compared to in the 5 minutes before the off, or compared to in play. A logical anchoring point is the moment in which the race starts, at which point BSP is also defined.

`TimeIndex` is created with time-zero defined at the second where `Inplay == True` for the first time for each `MarketId`, counting positively and negatively in either direction **in seconds**. 

In [12]:
# copying df
df_test = df.copy() # test on sample?

# converting to datetime
df_test['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# calculating inplay start for each race (assigning to new columns)
df_test['start_time'] = df_test['Time'].where(df_test['Inplay'] == True).groupby(df_test['MarketId']).transform('min')

# calculating difference between each time point and start time
df_test['time_dif'] = (df_test['Time'] - df_test['start_time']).astype('timedelta64[s]')

# converting time dif to minutes
df_test['time_dif_mins'] = round(df_test['time_dif'] / 60, 2)

In [13]:
df_test.head()
# test sample. MarketID : 1.168131, SelectionId 25545675

,Time,MarketId,Status,Inplay,MarketName,SelectionId,LastPriceTraded,TotalMatched,Sp,AdjFactor,RunnerStatus,MktTotalMatched,TradeSize,start_time,time_dif,time_dif_mins
0,2020-01-31 11:41:45.697,1.168131,OPEN,False,7f Hcap,25545675,NaN,NaN,NaN,5.95,ACTIVE,NaN,NaN,2020-02-01 12:35:45.684,-89640.0,-1494.0
1,2020-01-31 11:41:45.697,1.168131,OPEN,False,7f Hcap,25238892,NaN,NaN,NaN,30.53,ACTIVE,NaN,NaN,2020-02-01 12:35:45.684,-89640.0,-1494.0
2,2020-01-31 11:41:45.697,1.168131,OPEN,False,7f Hcap,24707172,NaN,NaN,NaN,7.01,ACTIVE,NaN,NaN,2020-02-01 12:35:45.684,-89640.0,-1494.0
3,2020-01-31 11:41:45.697,1.168131,OPEN,False,7f Hcap,24420205,NaN,NaN,NaN,8.45,ACTIVE,NaN,NaN,2020-02-01 12:35:45.684,-89640.0,-1494.0
4,2020-01-31 11:41:45.697,1.168131,OPEN,False,7f Hcap,26314798,NaN,NaN,NaN,33.12,ACTIVE,NaN,NaN,2020-02-01 12:35:45.684,-89640.0,-1494.0


In [14]:
df_test = df_test.astype({"MarketId": str, "SelectionId": str})

df_test.loc[(df_test['MarketId'] == '1.168130661') & (df_test['SelectionId'] == '25545675')][['Time', 'start_time', 'time_dif_mins']]

,Time,start_time,time_dif_mins
0,2020-01-31 11:41:45.697,2020-02-01 12:35:45.684,-1494.00
6,2020-01-31 11:41:46.694,2020-02-01 12:35:45.684,-1493.98
12,2020-01-31 11:41:47.524,2020-02-01 12:35:45.684,-1493.98
18,2020-01-31 11:41:48.611,2020-02-01 12:35:45.684,-1493.97
24,2020-01-31 11:41:49.607,2020-02-01 12:35:45.684,-1493.95
...,...,...,...
59310,2020-02-01 12:37:06.705,2020-02-01 12:35:45.684,1.35
59316,2020-02-01 12:37:07.610,2020-02-01 12:35:45.684,1.35
59322,2020-02-01 12:37:08.703,2020-02-01 12:35:45.684,1.38
59328,2020-02-01 12:39:11.618,2020-02-01 12:35:45.684,3.42


### How to separate prices into a 'testable' format?

- Take the average of of the prices at each given time point
- Filter for prices with trade size only over £100 ?
- Inplay time points separated depending on race length (this will be unknown in future/live races will have to be an estimate)


| SelectionId | >5 hours | 2-5 hours | 0.5-2 hours |0.5-0 hours|Inplay 'start'|Inplay 'middle| Inplay 'end|
|-------------|----------|-----------|-------------|-----------|--------------|--------------|------------|
|             |          |           |             |           |              |              |            |

- Have course, race_type and distance included too.
- Do we know back/lay side? Have a column for each?

In [15]:
df['MarketName'].value_counts()

7f Hcap                440650
1m4f Hcap              366493
2m7f Grd3 Hcap Hrd     340527
2m3f Cond Hcap Hrd     311950
1m Hcap                291854
1m7f Hcap Hrd          280224
3m Nov Hcap Chs        274606
2m NHF                 269273
2m1f Nov Hrd           240900
2m3f Nov Hrd           215582
2m1f Hcap Hrd          215088
1m4f Class Stks        214740
2m Mares Nov Hrd       191021
3m Hcap Chs            185148
1m Nov Stks            182434
5f Hcap                181530
1m1f Nov Stks          179720
2m2f Nov Hcap Chs      166032
1m5f Hcap              162668
2m Hcap                143451
2m4f Nov Hcap Hrd      143280
2m Nov Hrd             138429
2m Hcap Chs            134420
7f Sell Stks           130842
2m3f Hcap Hrd          128904
4m1f Hcap Chs          125960
2m7f Hcap Chs          121400
1m2f Hcap              102150
1m1f Claim Stks        101241
1m7f Listed             99192
1m4f Nov Stks           89832
3m2f Hcap Hrd           88900
2m4f Grd1 Nov Chs       87801
6f Hcap   

In [16]:
def extract_furlongs(market_name):
    '''
    Assuming distance is always stated 1st within 'MarketName', with space followed after.
    Distance given in format of furlongs, miles or both.
    8 furlongs in a mile.
    '''
    
    distance = market_name.split(' ')[0]
    
    if 'm' in distance:
        m = distance.split('m')[0]
        distance = distance.replace(m + 'm', '')
        
        if 'f' in distance:
            f = distance.split('f')[0]
            
            return (int(m) * 8) + int(f)

        return int(m) * 8
    
    else:
        f = distance.split('f')[0]
        
        return int(f)
    
    

In [17]:
extract_furlongs("2m Nov Chs")

16

In [22]:
def extract_race_type(market_name):
    if 'Hrd' in market_name:
        return 'Hurdle'
    if 'Chs' in market_name:
        return 'Chase'
    if 'NHF' in market_name:
        return 'NHF'
    else:
        return 'Flat'
    

In [23]:
extract_race_type("2m Nov Chs")

'Chase'

In [ ]:
# add in course info ? too difficult to add class?

In [19]:
# converting 'MarketName' to distance


# reading in time bins for distances
df_times = pd.read_csv('../race_time_eda/inplay_bins.csv', )  
    
# 

In [20]:
# how to format this?
# bins tim_dif_mins to buckets and then apply mean to each ?
# then drop unndeeded columns and transpose ?